# Black-Box Optimization of Design Parameters for Mixing Device

Mixing devices are essential in various fields, including chemical processes and food production. While their design and operating conditions significantly impact mixing efficiency (product quality), their determination often relies on experience and trial-and-error.

This tutorial introduces an approach to finding optimal conditions for five parameters involved in the device design using Black-Box Optimization (BBO) with Ising machine utilization. While the specific meanings of these parameters are not stated, they should be considered as "design choices" that influence the performance of the mixing device. As in actual design and control, we start from a state where "it is unclear what impact each design parameter has on the device's performance".

In this tutorial, we will evaluate the mixing device's performance (in this case, the degree of concentration variation after mixing) for different parameter combinations using the simulator. Understanding the details of the physical model or the simulator's contents is unnecessary. The primary focus of black-box optimization is on efficiently finding the best design parameters without knowing the shape and structure of the objective function.

Let's try black-box optimization by utilizing Ising machines in an engineering problem setting.

For a basic knowledge of black-box optimization based on machine learning and Ising machines used in this sample code, see "[Black Box Optimization with Quantum Annealing Ising Machines](https://amplify.fixstars.com/en/demo/fmqa_0_algebra)". For other black-box optimization examples, see [here](https://amplify.fixstars.com/en/demo#blackbox).

This sample program (optimal design parameters by black-box optimization) consists of the following:

- 1\. [Objective function overview](#obj)
  - 1\.1\. [Mixing simulator overview](#sim)
  - 1\.2\. [Implementing the black-box objective function](#bbfunc)
- 2\. [Black-box optimization implementation (integer variables)](#impl)
  - 2\.1\. [Defining the decision variable class](#var)
    - 2\.1\.1\. [What is domain wall encoding?](#encoding)
    - 2\.1\.2\. [Integer decision variable class `IntegerVariable`](#integer_variable)
  - 2\.2\. [FM model implementation](#fm)
  - 2\.3\. [Machine learning function implementation](#train)
  - 2\.4\. [Solver client setup](#client)
  - 2\.5\. [Optimization using the Ising machine](#opt)
  - 2\.6\. [Generating initial training data](#data)
- 3\. [Optimization of design parameters](#exec)
- 4\. [Evaluating optimization results and history](#eval)
  - 4\.1\. [Plotting results](#plot)
  - 4\.2\. [FMQA example run](#example)

---

\*In this online demo and tutorial environment, the continuous execution time is limited to about 20 minutes. If you expect the execution time to exceed 20 minutes, for example, when trying optimization by changing conditions, please copy this sample program to your local environment before execution. In that case, download the following libraries related to the MAS traffic simulator as appropriate and save them in the following directory structure.

├ [fmqa_5_mixing.ipynb](https://github.com/fixstars/amplify-examples/blob/main/notebooks/en/examples/fmqa_5_mixing.ipynb)（this program）  
└ utils/  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├ [\_\_init\_\_.py](https://github.com/fixstars/amplify-examples/blob/main/notebooks/en/examples/utils/__init__.py) （blank file）  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ [mixing.py](https://github.com/fixstars/amplify-examples/blob/main/notebooks/en/examples/utils/mixing.py)

<a id="obj"></a>

## 1\. Objective function overview

<a id="sim"></a>

### 1\.1\. Mixing simulator overview

In this tutorial, we use `MixingSimulator` as the stirring simulator. Below is a conceptual diagram of the stirrer.

![](../figures/fmqa_5_mixing/mixing.png)

The stirrer considered in the simulator has a total of five design parameters: `x0, x1, x2, x3, x4`. Inputting these parameters into the simulator determines the specifications of the stirrer. The initial state is set with a substance (black) added to the liquid, and stirring is simulated over a specific period based on these stirrer specifications. The result of the simulation is the degree of mixing (standard deviation of concentration).

Here, due to the constraints of the stirrer, the value ranges for these parameters are as follows:


In [ ]:
# Upper and lower limits for each design parameter
bounds: dict[str, tuple[int, int]] = {
    "x0": (2, 10),  # 2 <= x0 <= 10
    "x1": (5, 20),  # 5 <= x1 <= 20
    "x2": (0, 45),  # 0 <= x2 <= 45
    "x3": (1, 5),  # 1 <= x3 <= 5
    "x4": (1, 4),  # 1 <= x4 <= 4
}

Below are examples of how to use the `MixingSimulator` stirring simulator.

The `simulate()` method of `MixingSimulator` executes the stirring simulation and returns the standard deviation of the concentration `c_std` in the final concentration field. If the stirring is perfectly uniform, `c_std` will be zero. Additionally, the `plot_evolution()` method displays the time-series change of the concentration field.

The following simulation results show that the stirring process gradually progresses over time, and the standard deviation of the concentration distribution also decreases. Finally, a standard deviation of concentration of 0.031 is obtained, which indicates a relatively non-uniform concentration distribution.


In [ ]:
from utils.mixing import MixingSimulator

# Use the midpoint of each design parameter value range
x0, x1, x2, x3, x4 = tuple(int(0.5 * (v[0] + v[1])) for v in bounds.values())

# Initialize the simulator with the given parameter values
simulator = MixingSimulator(x0, x1, x2, x3, x4)

# Perform stirring simulation until time 500 and obtain the standard deviation of concentration, c_std
c_std = simulator.simulate(duration=500)

# Display results
print(f"{c_std=:.3f}")  # Standard deviation of concentration after stirring
simulator.plot_evolution(
    num_snaps=5
)  # Plot the time evolution of the concentration distribution during the stirring process

<a id="bbfunc"></a>

### 1\.2\. Implementing the black-box objective function

Based on the simulator `MixingSimulator` explained above, we will implement a black-box function. The black-box function `blackbox` below takes five design parameters as arguments, executes a simulation for a specific `duration`, and returns the result (the standard deviation of the final substance concentration).

The goal of this tutorial is to find a set of design parameters that minimizes the standard deviation of the concentration after mixing.


In [ ]:
def blackbox(x0: int, x1: int, x2: int, x3: int, x4: int) -> float:
    s = MixingSimulator(x0, x1, x2, x3, x4)
    c_std = s.simulate(duration=500)
    s.plot_evolution()
    print(f"{c_std=:.3f}")
    return c_std

<a id="impl"></a>

## 2\. Black-box optimization implementation (integer variables)

This section describes the program implementation of FMQA, a black-box optimization method that utilizes the Ising machine.

The processing flow of FMQA is described in [this tutorial](https://amplify.fixstars.com/en/demo/fmqa_0_algebra), but it is performed according to the following cycle.

In black-box optimization, including FMQA, the objective function is treated as a black box, allowing it to be generally applied without modifying the program implementation itself. Therefore, it is not always necessary to understand the program implementation of black-box optimization explained here.

![](../figures/fmqa_5_mixing/typical_flow_en.drawio.svg)

<a id="var"></a>

### 2\.1\. Defining the decision variable class

Since the Ising machine used during FMQA can directly handle only binary decision variables, appropriate encoding is necessary when considering non-binary decision variables such as integers and real numbers. In this tutorial, we consider integer decision classes with domain wall encoding and address non-binary decision variables.

<a id="encoding"></a>

#### 2\.1\.1\. What is domain wall encoding?

Domain-Wall Encoding is a method for converting discrete, non-binary variables (e.g., integer variables that can take values from 0 to k) into binary variables.

Assume an integer variable $x$ takes $(k+1)$ possible values in $\{0, 1, \dots, k\}$.

In Domain-Wall Encoding, values are represented using **$k$ binary variables** $q_1, q_2, \dots, q_k$ as follows:

- When $x = i$, the first $i$ elements of the variable sequence $\boldsymbol{q}$ are `1`, and the rest are `0`.
- In other words, the position where the switch from `1` to `0` occurs (the domain wall) represents the value of $x$.

#### Example: When $k = 4$

| Value of $x$ | Encoded bit string $\boldsymbol{q}$ |
| ------------ | ----------------------------------- |
| 0            | `[0, 0, 0, 0]`                      |
| 1            | `[1, 0, 0, 0]`                      |
| 2            | `[1, 1, 0, 0]`                      |
| 3            | `[1, 1, 1, 0]`                      |
| 4            | `[1, 1, 1, 1]`                      |

Thus, the representation of integer values is uniquely determined by the "wall position (domain wall)". The above explanation applies to integer variables with a minimum value of 0. Still, it can also be extended to integer variables with a non-zero minimum value or real variables by applying a discretization technique.

<a id="integer_variable"></a>

#### 2\.1\.2\. Integer decision variable class `IntegerVariable`

The `IntegerVariable` class, to be implemented below, efficiently represents integer decision variables by internally using the Amplify SDK's binary variables and applying domain wall encoding.

The core functionality of the `IntegerVariable` class is the conversion between integer values and their binary representations.

- `encode(self, x: int) -> np.ndarray`:
  This method converts the integer value `x` passed as an argument into a binary vector (NumPy array) based on the corresponding domain wall encoding. For example, encoding `x=3` for a variable with `bounds=(0, 5)` will return an array like `[1., 1., 1., 0., 0.]`.

- `decode(self, x: np.ndarray) -> int`:
  This method takes the binary variable results (NumPy array) computed by the Ising machine and decodes them back into their original integer values. Following the domain wall encoding, it reconstructs the original integer value by counting the number of ones in the array and adding the lower bound.

Additionally, the `IntegerVariable` class provides the following two properties:

- `constraint`:
  This property returns the domain wall constraint considered for this integer variable. In optimization, this constraint needs to be taken into account. This ensures that the Ising machine handles the associated binary variables (below `binary_variables`) according to the domain wall encoding.

- `binary_variables`:
  This property returns the vector of Amplify SDK binary variables that constitute this integer variable. It can be used when you want to reference the integer variable directly as a combination of binary variables.

The implementation below also considers `Variables`, which collectively manages multiple `IntegerVariable` objects.


In [ ]:
import amplify
import numpy as np


class IntegerVariable:
    """Class for integer decision variables for black-box optimization. Encodes and decodes integers using domain wall encoding."""

    def __init__(
        self, bounds: tuple[int, int], variable_generator: amplify.VariableGenerator
    ):
        self._bounds = bounds
        self._q = variable_generator.array("Binary", bounds[1] - bounds[0])
        self._constraint = amplify.domain_wall(self._q[::-1])

    @property
    def constraint(self) -> amplify.Constraint:
        """Return the constraints required for encoding integer decision variables."""
        return self._constraint

    @property
    def binary_variables(self) -> amplify.PolyArray:
        """Return the binary variable vector of the Amplify SDK that constitutes the integer decision variable."""
        return self._q

    def encode(self, x: int) -> np.ndarray:
        """Encode and binarize decision variable values."""
        if x < self._bounds[0] or x > self._bounds[1]:
            raise ValueError(f"x must be in {self._bounds}")
        ret = np.zeros(len(self._q))
        ret[0 : x - self._bounds[0]] = 1
        return ret

    def decode(self, x: np.ndarray) -> int:
        """Decode binary values to integer decision variable values."""
        if x.shape != self._q.shape:
            raise ValueError(f"x must be of shape {self._q.shape}")
        return x.sum() + self._bounds[0]


class Variables:
    """Class that manages a list composed of multiple integer decision variables."""

    def __init__(self, variable_list: list[IntegerVariable]):
        self._variable_list = variable_list

    def encode(self, x: list[int]) -> np.ndarray:
        """Encode and binarize decision variable values."""
        ret: list[int] = []
        for i, var in enumerate(self._variable_list):
            ret += var.encode(x[i]).tolist()
        return np.array(ret)

    def decode(self, x: np.ndarray) -> np.ndarray:
        """Decode binary values to integer decision variable values."""
        ret: list[int] = []
        ista = 0
        for var in self._variable_list:
            iend = ista + len(var.binary_variables)
            ret.append(var.decode(x[ista:iend]))
            ista = iend
        return np.array(ret, dtype=int)

    @property
    def constraints(self) -> amplify.ConstraintList:
        """Return the constraints required for encoding all integer decision variables."""
        return amplify.ConstraintList([var.constraint for var in self._variable_list])

    @property
    def binary_variables(self) -> amplify.PolyArray:
        """Return the binary variable vector of the Amplify SDK that constitutes all integer decision variables."""
        ret = np.array([])
        for var in self._variable_list:
            ret = np.concatenate((ret, var.binary_variables))  # type: ignore
        return amplify.PolyArray(ret.tolist())

    def __getitem__(self, i: int) -> IntegerVariable:
        return self._variable_list[i]

<a id="fm"></a>

### 2\.2\. FM model implementation

We will implement a `TorchFM` class using PyTorch to define the FM model, similar to how a standard machine learning model is typically constructed. The FM model is a machine learning model represented by the following polynomial. Here, $\boldsymbol{x}$ represents the variables, $d$ is a constant representing the length of the input to the black-box function, $\boldsymbol{v}$, $\boldsymbol{w}$, and $w_0$ are the model coefficients (weights and biases in machine learning terms), and $k$ is a hyperparameter representing the size of the parameters.

$$
\begin{aligned}
  f(\boldsymbol{x} | \boldsymbol{w}, \boldsymbol{v}) &=
  \underset{\color{red}{\mathtt{out\_linear}}}{\underline{ w_0 + \sum_{i=1}^d w_i x_i} } + \underset{\color{red}{\mathtt{out\_quadratic}}}{\underline{\frac{1}{2}
  \left[\underset{\color{red}{\mathtt{out\_1}}}{\underline{ \sum_{f=1}^k\left(\sum_{i=1}^d v_{i f} x_i\right)^2 }} - \underset{\color{red}{\mathtt{out\_2}}}{\underline{ \sum_{f=1}^k\sum_{i=1}^d v_{i f}^2 x_i^2 }} \right] }}
\end{aligned}
$$


In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Fix the random seed
seed = 0
rng = np.random.default_rng(seed)
torch.manual_seed(seed)


class TorchFM(nn.Module):
    def __init__(self, d: int, k: int):
        """Build the model

        Args:
            d (int): The size of the input vector
            k (int): Parameter k
        """
        super().__init__()
        self.d = d
        self.v = nn.Parameter(torch.randn((d, k)))
        self.w = nn.Parameter(torch.randn((d,)))
        self.w0 = nn.Parameter(torch.randn(()))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Takes input x and outputs an estimate of y."""
        out_linear = torch.matmul(x, self.w) + self.w0
        out_1 = torch.matmul(x, self.v).pow(2).sum(1)
        out_2 = torch.matmul(x.pow(2), self.v.pow(2)).sum(1)
        out_quadratic = 0.5 * (out_1 - out_2)

        out = out_linear + out_quadratic
        return out

    def get_parameters(self) -> tuple[np.ndarray, np.ndarray, float]:
        """Output parameters v, w, and w0."""
        np_v = self.v.detach().numpy().copy()
        np_w = self.w.detach().numpy().copy()
        np_w0 = self.w0.detach().numpy().copy()
        return np_v, np_w, float(np_w0)

<a id="train"></a>

### 2\.3\. Machine learning function implementation

Next, we will implement the function `train` for training the `TorchFM` model defined above. This process is also handled in the same way as standard machine learning. Still, as an essential performance metric for FM models in black-box optimization, we separately display the correlation coefficient between the predicted values of the trained model and the actual values.


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, random_split
from tqdm.auto import tqdm, trange
import copy


def train(
    x: np.ndarray,
    y: np.ndarray,
    model: TorchFM,
) -> None:
    """Train the FM model.

    Args:
        x (np.ndarray): Training data (input vectors)
        y (np.ndarray): Training data (output values)
        model (TorchFM): TorchFM model
    """

    # Number of iterations
    epochs = 2000
    # Model optimization function
    # optimizer = torch.optim.AdamW([model.v, model.w, model.w0], lr=0.1)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.1)  # type: ignore
    # Loss function
    loss_func = nn.MSELoss()

    # Prepare dataset
    x_tensor, y_tensor = (torch.from_numpy(x).float(), torch.from_numpy(y).float())

    dataset = TensorDataset(x_tensor, y_tensor)

    train_set, valid_set = random_split(dataset, [0.8, 0.2])
    if len(valid_set) == 0:
        valid_set = train_set
    train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
    valid_loader = DataLoader(valid_set, batch_size=8, shuffle=True)

    # Train the model
    min_loss = 1e18  # Save the minimum value of the loss function
    best_state = model.state_dict()  # Save the best model parameters of the model

    # Use the `tqdm` module to display progress instead of `range`.
    for _ in trange(epochs, leave=False):
        # Training process
        for x_train, y_train in train_loader:
            optimizer.zero_grad()
            pred_y = model(x_train)
            loss = loss_func(pred_y, y_train)
            loss.backward()
            optimizer.step()

        # Validation process
        with torch.no_grad():
            loss = 0
            for x_valid, y_valid in valid_loader:
                out_valid = model(x_valid)
                loss += loss_func(out_valid, y_valid)
            if loss < min_loss:
                # Save the parameters when the loss function value is updated
                best_state = copy.deepcopy(model.state_dict())
                min_loss = loss

    # Update the model with the trained parameters
    model.load_state_dict(best_state)

    # Display the correlation coefficient between the predicted values of the trained FM model and the true values
    print(
        f"corrcoef: {torch.corrcoef(torch.stack((model(x_tensor), y_tensor)))[0, 1].detach()}"
    )

<a id="client"></a>

### 2\.4\. Solver client setup

We will configure the Ising machine (solver client) for use in black-box optimization. In this tutorial, we use Fixstars Amplify Annealing Engine (Amplify AE). If running in a local environment, please enter the API token for Fixstars Amplify, which can be obtained for free.


In [ ]:
from amplify import AmplifyAEClient
from datetime import timedelta

# Set the solver client to Amplify AE
client = AmplifyAEClient()
# When running in a local environment, please uncomment and enter the Amplify AE access token
# client.token = "AE/xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
client.parameters.time_limit_ms = timedelta(milliseconds=10000)

<a id="opt"></a>

### 2\.5\. Optimization using the Ising machine

In this section, we define a function to optimize the trained FM model above. The procedure is as follows:

1. Based on the model coefficients of the trained FM model, construct a QUBO model mathematically equivalent to the FM model using Amplify SDK's decision variables.
2. Optimize the constructed QUBO model using an Ising machine.
3. Return the optimization results.

In [ ]:
from amplify import Model, solve, Poly

# Generate decision variables
gen = amplify.VariableGenerator()

var_list = [IntegerVariable(bounds=b, variable_generator=gen) for b in bounds.values()]
variables = Variables(var_list)

# The two lines above are essentially the same as the following process.
# x0 = IntegerVariable(bounds=bounds["x0"], variable_generator=gen)
# x1 = IntegerVariable(bounds=bounds["x1"], variable_generator=gen)
# x2 = IntegerVariable(bounds=bounds["x2"], variable_generator=gen)
# x3 = IntegerVariable(bounds=bounds["x3"], variable_generator=gen)
# x4 = IntegerVariable(bounds=bounds["x4"], variable_generator=gen)
# variables = Variables([x0, x1, x2, x3, x4])

# Constraints required for decision variable encoding
constraints = variables.constraints


def anneal(torch_model: TorchFM) -> np.ndarray:
    """Take the parameters of an FM model, find the x that yields the minimum value of the FM model described by those parameters."""

    # Get parameters v, w, and w0 from TorchFM
    v, w, w0 = torch_model.get_parameters()

    # Get binary decision variables of Amplify
    x = variables.binary_variables

    # Create a QUBO model (objective function) equivalent to the FM model
    out_linear = w0 + (x * w).sum()
    out_1 = ((x[:, np.newaxis] * v).sum(axis=0) ** 2).sum()  # type: ignore
    out_2 = ((x[:, np.newaxis] * v) ** 2).sum()
    objective: Poly = out_linear + (out_1 - out_2) / 2

    # Make Ampify model
    amplify_model = Model(objective, constraints)

    # Minimize (pass the constructed model and the solver client)
    result = solve(amplify_model, client)
    if len(result.solutions) == 0:
        raise RuntimeError("No solution was found.")

    # Return the input vector that minimizes the model (candidate for optimal design parameters)
    return x.evaluate(result.best.values).astype(int)

<a id="data"></a>

### 2\.6\. Generating initial training data

We evaluate the black-box function using randomly generated input vectors. The `n_0` input-output pairs obtained in this way are adopted as the initial training data.


In [ ]:
def generate_random_input() -> np.ndarray:
    x: list[int] = []
    for v_min, v_max in bounds.values():
        x.append(rng.integers(v_min, v_max + 1))
    return np.array(x)


def init_training_data(num_samples: int):
    # Generate n0 input values of length d using random numbers
    data: list[np.ndarray] = []
    for _ in range(num_samples):
        data.append(generate_random_input())
    x = np.array(data)

    # Remove duplicates in input values
    x = np.unique(x, axis=0)
    while x.shape[0] != num_samples:
        x = np.vstack((x, generate_random_input()))
        x = np.unique(x, axis=0)

    # Evaluate the blackbox function to obtain n0 outputs corresponding to the input values
    y = np.zeros(num_samples)
    for i in range(num_samples):
        y[i] = blackbox(*x[i])
    return x, y


n_0 = 10  # Number of initial training data
x, y = init_training_data(num_samples=n_0)

<a id="exec"></a>

## 3\. Optimization of design parameters

We will execute black-box optimization consisting of `n` cycles using the functions and classes implemented so far. In the code below, we set the number of times the objective function is evaluated to `n = 5`. This setting is for minimal operation confirmation in this demo/tutorial environment with a limited execution time. For execution conditions and examples of actual black-box optimization, please refer to "[FMQA Execution Examples](#example)".

For execution, we first encode the input vectors of the initial training data and convert them into binary values. While the FM model's training data considers the binarized `x_encoded`, it evaluates the black-box function using the decoded integer decision variables `x_hat_decoded`.


In [ ]:
# Number of FMQA cycles
n = 5  # For minimal operation check

# Encode the initial training data (x) into binary values
x_encoded = np.array([variables.encode(x[i]) for i in range(x.shape[0])])

# Iterate N times
# Display progress using the `tqdm` module instead of `range`
for i in trange(n):
    # Create machine learning model
    model = TorchFM(len(x_encoded[0]), k=10)

    # Execute model training
    train(x_encoded, y, model)

    # Get the input vector (encoded in binary) that gives the minimum value of the trained model
    x_hat = anneal(model)

    # If x_hat is identical to a sample in the training data, regenerate it randomly
    while (x_hat == x_encoded).all(axis=1).any():
        x_hat_random = generate_random_input()
        x_hat = variables.encode(x_hat_random.tolist())  # type: ignore
        print("deduplication")

    # Decode binary decision variable values to integer decision variable values
    x_hat_decoded = variables.decode(x_hat)

    # Evaluate the black-box function using the estimated input vector
    y_hat = blackbox(*x_hat_decoded)

    # Add the evaluated value to the dataset
    x_encoded = np.vstack((x_encoded, x_hat))
    y = np.append(y, y_hat)

    tqdm.write(f"FMQA cycle {i}: found y = {y_hat}; current best = {np.min(y)}")

<a id="eval"></a>

## 4\. Evaluating optimization results and history

<a id="plot"></a>

### 4\.1\. Plotting results

The following functions plot the transition of objective function evaluation values during the initial learning data generation process and the optimization cycle.


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot()

# Evaluation value of the black-box function for initial teacher data generation
ax.plot(
    range(-n_0 + 1, 1),
    y[:n_0],
    marker="o",
    linestyle="-",
    color="b",
)

# Evaluation value of the black-box function for FMQA cycles
ax.plot(
    range(1, n + 1),
    y[n_0:],
    marker="o",
    linestyle="-",
    color="r",
)

# History of updates to the minimum value of the objective function
ax.plot(
    range(-n_0 + 1, n + 1),
    [y[0]] + [min(y[:i]) for i in range(2, n_0 + n + 1)],
    linestyle="--",
    color="k",
)


ax.set_xlabel("number of iterations", fontsize=18)
ax.set_ylabel("f(x)", fontsize=18)
ax.tick_params(labelsize=18)
ax.set_yscale("log")
ax.set_ylim(1e-2, 2e-1)
plt.show()

print(f"best objective: {np.min(y):.3f}")
print(f"best solution: {variables.decode(x_encoded[np.argmin(y)]).tolist()}")

<a id="example"></a>

### 4\.2\. FMQA example run

In general, due to the nature of the heuristic algorithm employed by `AmplifyAEClient`, there is no absolute reproducibility in the obtained solutions. However, we present a typical execution result obtained when running the sample code below.

The figure below shows an example of execution when `n = 50`, illustrating the optimization history (the transition of black-box function values and the history of best solution updates).

![](../figures/fmqa_5_mixing/output_n50_history.png)

While there are increases and decreases in the objective function values in each optimization cycle, on average, the design parameters that stirr more uniformly are explored as the optimization cycles progress. In this execution example, a concentration standard deviation of 0.014 was obtained as the final best solution, and the stirring process at those design parameters is shown below.

![](../figures/fmqa_5_mixing/output_n50_simulation.png)